# Customer Clustering

A project to practice customer segmentation and to learn new clustering techniques

## Load packages

In [1]:
import pandas as np
import numpy as np
from matplotlib import pyplot as plt

## Loading Data

In [ ]:
df = pd.read